In [4]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/13 下午8:23
# @Author  : play4fun
# @File    : 47.2-使用SVM进行-手写数据OCR.py
# @Software: PyCharm

"""
47.2-使用SVM进行-手写数据OCR.py:
"""

import cv2
import numpy as np

SZ = 20
bin_n = 16  # Number of bins
affine_flags = cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR


# 使用方向梯度直方图Histogram of Oriented Gradients  HOG 作为特征向量
def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11'] / m['mu02']
    M = np.float32([[1, skew, -0.5 * SZ * skew], [0, 1, 0]])
    img = cv2.warpAffine(img, M, (SZ, SZ), flags=affine_flags)
    return img


# 计算图像 X 方向和 Y 方向的 Sobel 导数
def hog(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n * ang / (2 * np.pi))  # quantizing binvalues in (0...16)
    bin_cells = bins[:10, :10], bins[10:, :10], bins[:10, 10:], bins[10:, 10:]
    mag_cells = mag[:10, :10], mag[10:, :10], mag[:10, 10:], mag[10:, 10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)  # hist is a 64 bit vector
    return hist


# 最后 和前 一样 我们将大图分割成小图。使用每个数字的前 250 个作 为训练数据
#  后 250 个作为测试数据
img = cv2.imread('img/digits.png', 0)

cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
# First half is trainData, remaining is testData
train_cells = [i[:50] for i in cells]
test_cells = [i[50:] for i in cells]

deskewed = [map(deskew, row) for row in train_cells]
# deskewed = [deskew(row) for row in train_cells]
# deskewed = map(deskew, train_cells)
hogdata = [map(hog, row) for row in deskewed]
# hogdata = [hog(row) for row in deskewed]
# hogdata = map(hog, deskewed)

#trainData = np.float32(hogdata).reshape(-1, 64)
trainData = np.array(hogdata).reshape(-1, 64)
responses = np.float32(np.repeat(np.arange(10), 250)[:, np.newaxis])

svm = cv2.ml.SVM_create()
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setType(cv2.ml.SVM_C_SVC)
svm.setC(2.67)
svm.setGamma(5.383)
svm.train(trainData, cv2.ml.ROW_SAMPLE, responses)
svm.save('svm_data.dat')

deskewed = [map(deskew, row) for row in test_cells]
hogdata = [map(hog, row) for row in deskewed]
testData = np.float32(hogdata).reshape(-1, bin_n * 4)

result = svm.predict(testData)
mask = result == responses
correct = np.count_nonzero(mask)
print(correct * 100.0 / result.size)
# 94%


ValueError: cannot reshape array of size 50 into shape (64)

In [2]:
# -*- coding: utf-8 -*-
# @Time    : 2017/2/24 下午3:00
# @Author  : play4fun
# @File    : 48.2.2_仅有一个特征的数据.py
# @Software: PyCharm
%matplotlib
"""
48.2.2_仅有一个特征的数据.py:

输入参数
1. samples: 应 是 np.float32 类型的数据 每个特征应 放在一列。
2. nclusters(K): 聚类的最终数目。
3. criteria: 终止 代的条件。当条件满 时 算法的 代终止。它应 是 一个含有 3 个成员的元组 它们是 typw max_iter epsilon
4. attempts: 使用不同的 始标 来执 算法的次数。算法会 回紧密度 最好的标 。紧密度也会作为 出  回。
5. flags 用来 置如何 择 始 心。 常我们有两个 择 cv2.KMEANS_PP_CENTERS 和 cv2.KMEANS_RANDOM_CENTERS。

输出参数
1. compactness 紧密度  回每个点到相应 心的 离的平方和。
2. labels 标志数组 与上一节提到的代码相同  每个成员 标 为 0 1 等
3. centers 由聚类的中心组成的数组

"""

# 假 我们有一组数据 每个数据只有一个特征 1 维 。例如前 的 T 恤    我们只使用人们的  来决定 T 恤的大小。
# 我们先来产生一些 机数据 并使用 Matplotlib 将它们绘制出来。

import numpy as np
import cv2
from matplotlib import pyplot as plt

x = np.random.randint(25, 100, 25)
y = np.random.randint(175, 255, 25)
z = np.hstack((x, y))
z = z.reshape((50, 1))
z = np.float32(z)
plt.hist(z, 256, [0, 256]), plt.show()
# 现在我们有一个 度为 50 取值范围为 0 到 255 的向量z。我已经将向量z  重排 将它变成了一个列向量。
# 当每个数据含有多个特征是 会很有用。然后我们数据类型 换成 np.float32。

# exit(0)

##

# 现在我们使用KMeans函数。在之前我们应先置好终止条件。我的终止条件是算法执10次代或者精确度epsilon=1.0。

# Define criteria = ( type, max_iter = 10 , epsilon = 1.0 )
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# Set flags (Just to avoid line break in the code)

flags = cv2.KMEANS_RANDOM_CENTERS
# Apply KMeans
compactness, labels, centers = cv2.kmeans(z, 2, None, criteria, 10, flags)


# 返回值有紧密度compactness,标志和中心。在本例中我的到的中心是60和207。标志的数目与测数据的多少是相同的每个数据会标上01等。取决与它们的中心是什么。

A = z[labels == 0]
B = z[labels == 1]

# 现在我们可以根据它们的标志将把数据分两组。
# 现在将A组数用红色示
# 将B组数据用蓝色示,重心用黄色示。
# Now plot 'A' in red, 'B' in blue, 'centers' in yellow
plt.hist(A, 256, [0, 256], color='r')
plt.hist(B, 256, [0, 256], color='b')
plt.hist(centers, 32, [0, 256], color='y')
plt.show()


Using matplotlib backend: Qt5Agg


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/13 下午9:01
# @Author  : play4fun
# @File    : 48.2.2-含有多个特征的数据.py
# @Software: PyCharm
%matplotlib
"""
48.2.2-含有多个特征的数据.py:
身高
体重

"""

# 在前 的 T 恤例子中我们只考 了   现在我们也把体 考  去 也 就是两个特征。
# 在前一节我们的数据是一个单列向 。每一个特征 排列成一列 每一  对应一个测 样本。
# 在本例中我们的测 数据 应 50x2 的向  其中包含 50 个人的  和 体 。第一列对应与   第二列对应与体 。第一 包含两个元素 第一个 是第一个人的   
#第二个是第一个人的体 。剩下的 对应与其他人的   和体 。

import numpy as np
import cv2
from matplotlib import pyplot as plt

X = np.random.randint(25, 50, (25, 2))
Y = np.random.randint(60, 85, (25, 2))
Z = np.vstack((X, Y))

# convert to np.float32
Z = np.float32(Z)
# define criteria and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret, label, center = cv2.kmeans(Z, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Now separate the data, Note the flatten()
A = Z[label.ravel() == 0]
B = Z[label.ravel() == 1]

# Plot the data
plt.scatter(A[:, 0], A[:, 1])
plt.scatter(B[:, 0], B[:, 1], c='r')
plt.scatter(center[:, 0], center[:, 1], s=80, c='y', marker='s')
plt.xlabel('Height'), plt.ylabel('Weight')
plt.show()


In [3]:
# -*- coding: utf-8 -*-
# @Time    : 2017/2/24 下午3:17
# @Author  : play4fun
# @File    : 48.2.3_颜色量化.py
# @Software: PyCharm

"""
48.2.3_颜色量化.py:
"""

# 颜色量化就是减少图片中颜色数目的一个过程。为什么 减少图片中的  色呢 减少内存消耗 有些 备的 源有  只能显示很少的 色。在 种情 况下就     
#色 化。我们使用 K 值聚类的方法来   色 化。
# 没有什么新的知   介绍了。现在有 3 个特征 R G B。所以我们   把图片数据变形成 Mx3 M 是图片中像素点的数目 的向 。聚类完成后  我们用聚
#类中心值替换与其同组的像素值  样结果图片就只含有指定数目的  色了。下 是代码
# -*- coding: utf-8 -*-

import numpy as np
import cv2

img = cv2.imread('img/home.jpg')
# img = cv2.imread('../data/opencv_logo.png')
Z = img.reshape((-1, 3))
# convert to np.float32
Z = np.float32(Z)


# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# K = 8
# K = 3
K = 14
ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)


# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

cv2.imshow('res2', res2)
cv2.waitKey(10000)
cv2.destroyAllWindows()


In [5]:
# -*- coding: utf-8 -*-
# @Time    : 2017/2/24 下午3:17
# @Author  : play4fun
# @File    : 48.2.3_颜色量化.py
# @Software: PyCharm

"""
48.2.3_颜色量化.py:
"""

#  色 化就是减少图片中 色数目的一个 程。为什么 减少图片中的  色呢 减少内存消耗 有些 备的 源有  只能显示很少的 色。在 种情 况下就     色 化。
#我们使用 K 值聚类的方法来   色 化。
# 没有什么新的知   介绍了。现在有 3 个特征 R G B。所以我们   把图片数据变形成 Mx3 M 是图片中像素点的数目 的向 。聚类完成后  我们用聚类中心
#值替换与其同组的像素值  样结果图片就只含有指定数目的  色了。下 是代码
# -*- coding: utf-8 -*-

import numpy as np
import cv2

img = cv2.imread('img/home.jpg')
# img = cv2.imread('../data/opencv_logo.png')
Z = img.reshape((-1, 3))
# convert to np.float32
Z = np.float32(Z)


# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 8
# K = 3
# K = 14
ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

#分离颜色
for y in range(len(center)):
    a1 = []
    for i,x in enumerate(label.ravel()):
        if x==y:
            a1.append(list(Z[i]))
        else:
            a1.append([0,0,0])
    a2=np.array(a1)
    a3=a2.reshape((img.shape))
    cv2.imshow('res2'+str(y), a3)

#最大的色块



# # Now convert back into uint8, and make original image
# center = np.uint8(center)
# res = center[label.flatten()]
# res2 = res.reshape((img.shape))

# cv2.imshow('res2', res2)
# cv2.imshow('res2', a3)
cv2.waitKey(10000)
cv2.destroyAllWindows()


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/13 下午9:57
# @Author  : play4fun
# @File    : 2-fastNlMeansDenoisingMulti.py
# @Software: PyCharm

"""
2-fastNlMeansDenoisingMulti.py:
"""

import numpy as np
import cv2
from matplotlib import pyplot as plt

cap = cv2.VideoCapture('../data/vtest.avi')
# create a list of first 5 frames
img = [cap.read()[1] for i in range(5)]
# convert all to grayscale
gray = [cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in img]
# convert all to float64
gray = [np.float64(i) for i in gray]


# create a noise of variance 25
noise = np.random.randn(*gray[1].shape) * 10
# Add this noise to images
noisy = [i + noise for i in gray]
# Convert back to uint8
noisy = [np.uint8(np.clip(i, 0, 255)) for i in noisy]

# Denoise 3rd frame considering all the 5 frames
dst = cv2.fastNlMeansDenoisingMulti(noisy, 2, 5, None, 4, 7, 35)


plt.subplot(131), plt.imshow(gray[2], 'gray')
plt.subplot(132), plt.imshow(noisy[2], 'gray')
plt.subplot(133), plt.imshow(dst, 'gray')
plt.show()


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/13 下午9:56
# @Author  : play4fun
# @File    : 1-fastNlMeansDenoisingColored.py
# @Software: PyCharm

"""
1-fastNlMeansDenoisingColored.py:
"""

import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('die.png')
img = cv2.cvtColor(img, code=cv2.COLOR_BGR2RGB)

dst = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
# dst2=cv2.cvtColor(dst,code=cv2.COLOR_BGR2RGB)

plt.subplot(121), plt.imshow(img)
plt.subplot(122), plt.imshow(dst)
# plt.subplot(122), plt.imshow(dst2)
plt.show()


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/13 下午11:43
# @Author  : play4fun
# @File    : HDR.py
# @Software: PyCharm

"""
HDR.py:
http://docs.opencv.org/3.2.0/d2/df0/tutorial_py_hdr.html

了解如何从曝光序列生成和显示HDR图像。
使用曝光融合合并曝光序列。

高动态范围成像（HDRI或HDR）是一种用于成像和摄影的技术，可以重现比标准数字成像或摄影技术更大的动态光度范围。虽然人眼可以调整到广泛的光线条件，
但大多数成像设备每通道使用8位，因此我们仅限于256级。当我们拍摄现实世界的场景时，明亮的地区可能曝光过度，而黑暗的区域可能曝光不足，所以我们
无法使用单次曝光拍摄所有细节。HDR图像与每个通道使用超过8位（通常为32位浮点值）的图像一起使用，允许更宽的动态范围。

有不同的获取HDR图像的方法，但最常见的是使用不同曝光值拍摄的场景的照片。要组合这些曝光，了解您的相机的响应功能是有用的，并且有算法来估计它。
合并HDR图像后，必须将其转换回8位才能在通常的显示屏上进行查看。这个过程叫做tonemapping。当场景或相机的对象在拍摄之间移动时，会出现附加的复
杂性，因为具有不同曝光的图像应该被注册和对齐。

在本教程中，我们展示了两种算法（Debvec，Robertson）从曝光序列生成和显示HDR图像，并展示了一种称为曝光融合（Mertens）的替代方法，它产生低动
态范围图像，不需要曝光时间数据。此外，我们估计对于许多计算机视觉算法具有重要价值的相机响应函数（CRF）。HDR管道的每一步都可以使用不同的算法
和参数来实现，因此请参考参考手册来查看。
"""

import cv2
import numpy as np

# 第一阶段只是将所有图像加载到列表中。此外，我们将需要常规HDR算法的曝光时间。注意数据类型，因为图像应为1通道或3通道8位（np.uint8），曝光时间
#需要为float32，以秒为单位。
# Loading exposure images into a list
img_fn = ["1tl.jpg", "2tr.jpg", "3bl.jpg", "4br.jpg"]
img_list = [cv2.imread(fn) for fn in img_fn]
exposure_times = np.array([15.0, 2.5, 0.25, 0.0333], dtype=np.float32)

# Merge exposures to HDR image
# 在这个阶段，我们将曝光序列合并成一个HDR图像，显示了我们在OpenCV中的两种可能性。第一种方法是Debvec，第二种是Robertson。请注意，HDR图像的类
#型为float32，而不是uint8，因为它包含所有曝光图像的完整动态范围。

merge_debvec = cv2.createMergeDebevec()
hdr_debvec = merge_debvec.process(img_list, times=exposure_times.copy())
merge_robertson = cv2.createMergeRobertson()
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())

# Tonemap HDR image
# 我们将32位浮点HDR数据映射到范围[0..1]。实际上，在某些情况下，值可能大于1或低于0，所以注意我们以后不得不剪切数据，以避免溢出。
tonemap1 = cv2.createTonemapDurand(gamma=2.2)
res_debvec = tonemap1.process(hdr_debvec.copy())
tonemap2 = cv2.createTonemapDurand(gamma=1.3)
res_robertson = tonemap2.process(hdr_robertson.copy())

# Exposure fusion using Mertens
# 这里我们展示了一种可以合并曝光图像的替代算法，我们不需要曝光时间。我们也不需要使用任何tonemap算法，因为Mertens算法已经给出了[0..1]范围内的结果。
merge_mertens = cv2.createMergeMertens()
res_mertens = merge_mertens.process(img_list)

# Convert datatype to 8-bit and save
# 为了保存或显示结果，我们需要将数据转换为[0..255]范围内的8位整数。
res_debvec_8bit = np.clip(res_debvec * 255, 0, 255).astype('uint8')
res_robertson_8bit = np.clip(res_robertson * 255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens * 255, 0, 255).astype('uint8')

cv2.imwrite("ldr_debvec.jpg", res_debvec_8bit)
cv2.imwrite("ldr_robertson.jpg", res_robertson_8bit)
cv2.imwrite("fusion_mertens.jpg", res_mertens_8bit)

exit(0)

# Estimate camera response function (CRF)
# 相机响应功能（CRF）给出了场景辐射度与测量强度值之间的连接。如果在一些计算机视觉算法中非常重要，包括HDR算法，CRF。这里我们估计反相机响应函数并将其用于HDR合并。
cal_debvec = cv2.createCalibrateDebevec()
crf_debvec = cal_debvec.process(img_list, times=exposure_times)
hdr_debvec = merge_debvec.process(img_list, times=exposure_times.copy(), response=crf_debvec.copy())
cal_robertson = cv2.createCalibrateRobertson()
crf_robertson = cal_robertson.process(img_list, times=exposure_times)
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy(), response=crf_robertson.copy())
